In [1]:
import torch
import glob
import tqdm

import numpy as np
import torch.nn as nn
import torch.nn.functional as F

from utilities import utils, train_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from torch.utils.data import DataLoader, TensorDataset
from pythae.models import AE, AEConfig
from pythae.trainers import BaseTrainerConfig
from pythae.pipelines.training import TrainingPipeline
from pythae.models.nn import BaseEncoder, BaseDecoder
from pythae.models.base.base_utils import ModelOutput

In [19]:
paths = sorted(glob.glob('./data/environmental_embeddings_0001/0001/*.msgpack'))
device = train_utils.get_device()

X = []
for path in tqdm.tqdm(paths):
    pos_emb, neg_emb = utils.read_embedding_data(path)
    X.append(pos_emb)
X = np.concatenate(X, axis=0)
X = X.reshape(-1, 768)

Xtr, Xvl = train_test_split(X, test_size=0.2, random_state=42)

Xtr_scl = normalize(Xtr, norm='l2', axis=1)
Xvl_scl = normalize(Xvl, norm='l2', axis=1)

train_data = torch.tensor(Xtr_scl, dtype=torch.float32)
val_data = torch.tensor(Xvl_scl, dtype=torch.float32)

train_dataset = TensorDataset(train_data)
val_dataset = TensorDataset(val_data)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)

100%|██████████| 1000/1000 [00:05<00:00, 197.27it/s]


In [20]:
class Encoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(Encoder, self).__init__()
        self.input_dim = input_dim
        self.latent_dim = latent_dim

        self.encoder_layers = nn.Sequential(
            nn.Linear(self.input_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, self.latent_dim),
        )

    def forward(self, x):
        x = self.encoder_layers(x)

        return x
    
class Decoder(BaseDecoder):
    def __init__(self, input_dim, latent_dim):
        super(Decoder, self).__init__()
        self.input_dim = input_dim
        self.latent_dim = latent_dim

        self.decoder_layers = nn.Sequential(
            nn.Linear(self.latent_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, self.input_dim),
            nn.Tanh()
        )

    def forward(self, z):
        x = self.decoder_layers(z)

        return x
    
class AutoEncoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(AutoEncoder, self).__init__()

        self.encoder = Encoder(input_dim, latent_dim)
        self.decoder = Decoder(input_dim, latent_dim)

    def forward(self, x):
        self.z = self.encoder(x)
        recon = self.decoder(self.z)

        return recon

In [21]:
model = AutoEncoder(input_dim=768, latent_dim=128)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, threshold=0.01)
criterion = torch.nn.MSELoss(reduction='sum')
model = model.to(device)

In [22]:
train_utils.train_loop(
    train_loader=train_loader,
    val_loader=val_loader,
    model=model,
    criterion=criterion,
    device=device,
    optimizer=optimizer,
    epochs=500,
    save_path='experiments/fc_exp3/',
    sparsity_penalty_weight=0.001,
    scheduler=None
)

val loss: 0.417: 100%|██████████| 121/121 [00:00<00:00, 798.76it/s]


Weight saved: epoch 0
Epoch 0	Train Loss: 0.519  Val Loss: 0.401



val loss: 0.375: 100%|██████████| 121/121 [00:00<00:00, 848.39it/s]


Weight saved: epoch 1
Epoch 1	Train Loss: 0.381  Val Loss: 0.360



val loss: 0.348: 100%|██████████| 121/121 [00:00<00:00, 916.85it/s]


...

Epoch 149	Train Loss: 0.115  Val Loss: 0.130



val loss: 0.126: 100%|██████████| 121/121 [00:00<00:00, 808.73it/s]


Epoch 150	Train Loss: 0.114  Val Loss: 0.130



train loss: 0.130:  76%|███████▌  | 367/482 [00:01<00:00, 236.32it/s]


KeyboardInterrupt: 

In [24]:
model.eval()
with torch.no_grad():
    recon = model(val_data.to(device))

criterion = torch.nn.MSELoss(reduction='sum')
criterion(val_data.to(device), recon) / 15400

tensor(0.1290, device='cuda:0')